In [5]:
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl, numpy as np
from pathlib import Path
from torch import tensor
from fastcore.test import test_close
torch.manual_seed(42)

mpl.rcParams['image.cmap'] = 'gray'
torch.set_printoptions(precision=2, linewidth=125, sci_mode=False)
np.set_printoptions(precision=2, linewidth=125)

path_data = Path('data')
path_gz = path_data/'mnist.pkl.gz'
with gzip.open(path_gz, 'rb') as f: ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
x_train, y_train, x_valid, y_valid = map(tensor, [x_train, y_train, x_valid, y_valid])


In [6]:
n, m  = x_train.shape
c = y_train.max() + 1
nh = 50
w1 = torch.randn(m, nh)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)
b2 = torch.zeros(1)
n,m,c

(50000, 784, tensor(10))

### Model

In [7]:
# recitfied linear (y = x > 0 ? x : 0)
class Relu():
    def forward(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)
        return self.out
    
    def backward(self): 
        self.inp.g = (self.inp > 0).float() * self.out.g


In [21]:
class Lin():
    def __init__(self, 
                 w, # weights
                 b # biases
                 ): self.w,self.b = w,b
    
    def forward(self, inp):
        self.inp = inp
        self.out = inp @ self.w + self.b
        return self.out
    
    def backward(self): 
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = self.inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

In [22]:
class Mse():
    def forward(self, inp, targ):
        self.inp, self.targ = inp, targ
        self.out = (self.inp[:,0]-targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [23]:
class Model(): 
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1, b1), Relu(),  Lin(w2, b2)]
        self.loss = Mse()
        
    def forward(self, x, targ):
        for l in self.layers: x = l.forward(x)
        return self.loss.forward(x, targ)
    
    
    def backward(self): 
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [24]:
model = Model(w1, b1, w2, b2)
loss = model.forward(x_train, y_train)

In [25]:
model.backward()